<a href="https://colab.research.google.com/github/ritwickbhargav80/autogen-agent-playground/blob/main/Dual%20Agents/News%20Retrieval%20%26%20Summarization/AutoGen_Dual_Agents_%5BNews_Retrieval_%26_Summarization%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing AutoGen

AutoGen supports Python version >= 3.8 and < 3.13. In the below cell we are installing `pyautogen` through pip.

In [ ]:
!pip install pyautogen

# Setting Up The Environment

Here we're using the configuration of GPT-3.5 Turbo model from Azure OpenAI and storing the News API key in environment variable.

GPT Configuration is taken from the Google Colab Secrets.

> **NOTE:** We want to prevent the sensitive News API key from directly being exposed to the LLM model. However, we want code generated by LLM to still have access to the News API Key. Thus, environmental variables are best suited.

In [ ]:
from google.colab import userdata
import os

config_list = [
    {
        "model": "gpt-4",
        "base_url": userdata.get("AZURE_OPENAI_ENDPOINT"),
        "api_key": userdata.get("AZURE_OPENAI_API_KEY"),
        "api_type": "azure",
        "api_version": "2024-02-15-preview"
    }
]

os.environ["NEWS_API_KEY"] = "<NEWSORG_API_KEY>"

# Creating AI Agents

This is where our agents gets defined. Here we are creating two agents:

1. **AssistantAgent:** It acts as an AI model. It can use LLM to generate responses. It can even write code, check the results and suggest fixes.
2. **UserProxyAgent:** It can mimic the user's interaction. Here, we have provided the UserProxyAgent the capability to perform code execution. With options like "code_execution_config" and "llm_config", you can regulate code execution and LLM utilization for response generation.

In [ ]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.coding import LocalCommandLineCodeExecutor

llm_config = {
      "cache_seed": 41,
      "config_list": config_list,
      "temperature": 0
  }

# defining initalizer agent to initiate chat and also responsible for executing code generated by coder agent
initializer_agent = UserProxyAgent(
    name="Initializer",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={ "executor": LocalCommandLineCodeExecutor(work_dir="coding") },
    default_auto_reply="Reply 'TERMINATE' if the task is done."
)

# defining coder agent for generating code
coder_agent = AssistantAgent(
    name="Coder",
    llm_config=llm_config,
)

# defining summarizer agent to summarize based on the user input and previous agent chat output
summarizer_agent = AssistantAgent(
    name="Summarizer",
    llm_config=llm_config,
    system_message="""
      You are an experienced technology news editor with expertise in analyzing and synthesizing technical information. Your task is to:

      1. Carefully review the provided content
      2. Create concise, well-structured summary that highlight:
        - Key technological breakthroughs or announcements
        - Impact on the industry or users
        - Notable companies, products, or researchers involved
        - Timeline and context if relevant

      Write clear, engaging summary that help readers quickly understand the most important tech news developments.
      Focus on accuracy, relevance, and readability. The newspaper article should be informative enough and concise enough for busy professionals.

      Format your output as organized news article with clear headline and essential details.
      Generate a single organized news article with a well-curated catchy headline followed by a two-paragraph summary that synthesizes all 10 news articles into one cohesive overview.
      The summary should contain all the essential information a professional needs to stay informed about the latest tech developments.
      Format the output in Markdown with just the main headline and summarized content paragraphs - no subheadings or additional structure.

      Do not provide any supporting text.
    """,
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    default_auto_reply="Reply 'TERMINATE' if the task is done."
)

# Setting Up Multi-Agent Collaboration

This is where we setup the multi-agent collaboration using AutoGen's GroupChat functionality. We create a GroupChat with our three specialized agents (initializer, coder, and summarizer), establish a GroupChatManager to orchestrate their interactions, and initiate the conversation with instructions to retrieve and summarize tech news from TechCrunch using the NewsAPI.

In [ ]:
from autogen import GroupChat, GroupChatManager

# defining group chat and group chat manager
groupchat = GroupChat(agents=[initializer_agent, coder_agent, summarizer_agent], messages=[], max_round=10)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# initiation of the chat by initializer user proxy agent
chat_result = initializer_agent.initiate_chat(
    manager,
    message="""
        Do you know about newsapi.org?
        Can you use that to retrieve the top 10 tech news headlines and the content from techcrunch.com?
        Your API KEY for newsapi.org is stored in os.environ["NEWS_API_KEY"].

        INSTRUCTIONS:
        - Write code to print just the 10 tech news headlines and content
        - Execute the code which will result in the output with headlines, content getting printed
        - Summarize the tech news in in a professional, concise format and focus on keeping digestible insights
        - After summarizing, say "TERMINATE" and then stop responding
    """
)

Initializer (to chat_manager):


        Do you know about newsapi.org?
        Can you use that to retrieve the top 10 tech news headlines and the content from techcrunch.com?
        Your API KEY for newsapi.org is stored in os.environ["NEWS_API_KEY"].

        INSTRUCTIONS:
        - Write code to print just the 10 tech news headlines and content
        - Execute the code which will result in the output with headlines, content getting printed
        - Summarize the tech news in in a professional, concise format and focus on keeping digestible insights
        - After summarizing, say "TERMINATE" and then stop responding
    

--------------------------------------------------------------------------------

Next speaker: Coder

Coder (to chat_manager):

To retrieve the top 10 tech news headlines and content from TechCrunch using the NewsAPI, I will write Python code that utilizes the `requests` library to interact with the NewsAPI endpoint. The code will fetch the data, extract the

# Displaying the Final News Summary

This is where we extract and display the final generated news summary from the multi-agent conversation.

In [ ]:
from IPython.display import Markdown, display

md_content = chat_result.chat_history[-1]['content'].replace("TERMINATE", "")
display(Markdown(md_content))

**AI Innovations, Supersonic Travel, and IPO Milestones Shape Tech Landscape**

Recent developments in technology highlight transformative advancements across AI, transportation, and healthcare. Superblocks CEO Brad Menezes emphasizes the potential of AI system prompts to inspire billion-dollar startup ideas, while OpenAI collaborates with startups to enhance AI engines. At TechCrunch Sessions: AI, companies like Fluix AI and Narada AI showcased their innovations, reflecting the growing momentum in generative AI applications. Meanwhile, Oracle's MySQL HeatWave is enabling scalable AI solutions for developers, and Meta's CTO predicts 2025 as a pivotal year for augmented and virtual reality breakthroughs.

In transportation, President Trump fast-tracked supersonic travel by lifting a decades-old ban, signaling a shift in aviation policy. Rivian's collaboration with Jony Ive's LoveFrom and Uber's next-gen strategies further underscore mobility advancements. On the healthcare front, Omada Health's IPO marks a healthier market trend, avoiding down-round pitfalls. Additionally, VivaTech spotlighted visionary startups tackling global challenges, and KiranaPro's data breach raises cybersecurity concerns. These developments collectively illustrate the dynamic evolution of technology across industries.



# Voila!

Congratulations on creating your first groupchat of AutoGen agents to retrieve news from techcrunch.com and getting it summarized into professional tech briefings!